In [54]:
import requests
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [55]:
#goal = album or track
def geturl(goal,artist,title):
    url='https://ws.audioscrobbler.com/2.0/?method={}.getinfo&artist={}&api_key=cbe33f95f9dc5877b258dea8d58c9e14&{}&extended=1&format=json'
    artist = artist.replace(' ', '+')
    title = goal+'='+title.replace(' ', '+')
    url=url.format(goal,artist,title)
    rec=None
    try:
        rec=requests.get(url).json()
    except:
        pass
    return rec

In [56]:
df=pd.DataFrame()  #df is the full, with all the tracks
#SEARCH ALBUM |ARTIST, ALBUM|
df['Artist']= None
df['Album']= None
df['TrackNum']= None
df['Title']= None
# df['AlbumRelease']= None
df['Duration']=None
df['Tag1']=None
df['Tag2']=None

#SEARCH TRACK |ARTIST, SONG|
df['Listeners']= None
df['PlayCount']= None
df['TrackRelease']=None

# #SEACH TRACKS AS ALBUMS |ARTIST,SONG|
# df['SListeners']=None
# df['SPlayCount']=None
# df ['SRelease']=None
# df['STracks']=None

In [57]:
tracklist=pd.DataFrame() #the album here

tracklist['Artist']=None
tracklist['Album']=None
tracklist['TrackNum']=None
tracklist['Title']=None
tracklist['Duration']=None
# tracklist['AlbumRelease']=None
tracklist['Tag1']=None
tracklist['Tag2']=None
tracklist['Tracks']=None
counter=0

In [58]:
def getalbum(artist,title):
    tracklist=pd.DataFrame()
    json=geturl('album',artist,title)
    print(json)
    album = []
    artist = []
    released8 = []
    tracknum = []
    tracktitle = []
    duration = []
    twotags = []
    tag1 = []
    tag2 = []
    length = []
    
    try:
        item =  json['album']['tags']['tag']
    except:
        pass
    
    try:
        if item[0]['name'].isdigit() != True:
            twotags.append((item[0]['name']))
        else:
            twotags.append((item[1]['name']))
            twotags.append((item[2]['name']))
    except:
        pass
    
            
    try:
        for item in json['album']['tracks']['track']:
            tracktitle.append(item['album']['tracks']['track']['name'])
            duration.append(item['duration'])
            tracknum.append(item['@attr']['rank'])
            artist.append(item['artist']['name'])
            album.append(json['album']['name'])
            try: 
                released8.append(json['album']['wiki']['published'])
            except:
                pass
            try:
                tag1.append(twotags[0])
            except:
                pass
            try:
                tag2.append(twotags[1])
            except:
                pass
            length.append(len(json['album']['tracks']['track']))
    except:
        pass
    tracklist['Artist']=artist
    tracklist['Album']=album
    
    try:
        print(json['album']['name'])
        
    except:
        pass
    tracklist['TrackNum']=tracknum
    tracklist['Title']=tracktitle
    tracklist['Duration']=duration
    try:
        tracklist['Tag1']=tag1
    except:
        pass
    try:
        tracklist['Tag2']=tag2
    except:
        pass
    tracklist['Tracks']=length
    return tracklist

In [59]:
def getalbum(artist,title):
    try:
        json=geturl('album',artist,title)
        artist = json['album']['artist']
        album = json['album']['name']
        tracks_data = json['album']['tracks']['track']
        tracks_row = [[track['name'], track['duration'], track['@attr']['rank'], getsong(artist, track['name'])] for track in tracks_data]
        row_dict = []
        for row in tracks_row:
            row_dict.append({'artist': artist, 'album': album, 'track-title':row[0], 'duration': row[1], 
                            'track_no': row[2], 'listeners': row[3][0], 'playcount': row[3][1], 'single_release': row[3][2]})
        tracklist = pd.DataFrame(row_dict)
        return tracklist
    except:
        print (artist, title)

In [10]:
df=pd.DataFrame()

In [60]:
def getsong(artist,title):
    trackrelease=None
    try:
        song=geturl('track',artist,title)
    except:
        pass
    try:
        listeners=(song['track']['listeners'])
    except:
        pass
    try:
        playcount=(song['track']['playcount'])
    except:
        pass
    try:
        trackrelease=(song['track']['wiki']['published'])
    except:
        pass 
    return listeners, playcount,trackrelease

In [61]:
from scrape_billboard import *

In [2]:
albums2018 = scrape_billboard_albums(billboard_url)
len(albums2018)

200

In [63]:
for i in missed:
    project = getalbum(i[0],i[1])
    df = pd.concat([df,project])

Khalid American Teen
Juice WRLD Goodbye %26 Good Riddance
Luke Combs This One's For You
Kane Brown Kane Brown
G-Eazy The Beautiful %26 Damned
Chris Brown Heartbreak On A Full Moon
Drake Views
Rae Sremmurd SR3MM
Miguel War %27 Leisure
Wiz Khalifa Rolling Papers 2
The Chainsmokers Memories%2E%2E%2EDo Not Open
DJ Khaled Grateful
Trippie Redd A Love Letter To You
Drake If you're reading this it's too late
Charlie Puth Voicenotes
U2 Songs Of Experience


In [65]:
df.to_csv('./lastfmmissed.csv')

In [62]:
missed =[('Travis $cott', 'ASTROWORLD'), ('Migos', 'Culture II'), 
 ('Khalid','American Teen'),('Post Malone','Beerbongs %26 Bentleys'),
 ('Juice WRLD', 'Goodbye %26 Good Riddance'), ('Luke Combs', "This One's For You"),
 ('Lil%27 Wayne', 'Tha Carter V'),('Kane Brown', 'Kane Brown'),
 ('G-Eazy', 'The Beautiful %26 Damned'),('Chris Brown', 'Heartbreak On A Full Moon'),
 ('21 Savage', 'Without Warning'), ('Drake','Views'),
 ('Shawn Mendes','Shawn Mendes'),
 ('Kendrick Lamar', 'good+kid,+m.A.A.d+city'),
 ('Lil Baby', 'Drip Harder'), ('Rae Sremmurd', 'SR3MM'),
 ('Miguel', 'War %27 Leisure'), ('Jhené+Aiko', 'TRIP'),
 ('Wiz Khalifa', 'Rolling Papers 2'), ('The Chainsmokers', "Memories%2E%2E%2EDo Not Open"),
 ('DJ Khaled', 'Grateful'),('Trippie Redd', 'A Love Letter To You'),('Kanye West', 'The Life Of Pablo'),
 ('Drake',"If you're reading this it's too late"),('Charlie Puth', 'Voicenotes'),
 ('U2', 'Songs Of Experience')]

In [64]:
len(missed)

26

In [50]:
geturl('album','Drake',"If you're reading this it's too late")

{'album': {'name': "If You're Reading This It's Too Late",
  'artist': 'Drake',
  'url': 'https://www.last.fm/music/Drake/If+You%27re+Reading+This+It%27s+Too+Late',
  'image': [{'#text': 'https://lastfm-img2.akamaized.net/i/u/34s/9f25933143434b6ccb25d7214133f34e.png',
    'size': 'small'},
   {'#text': 'https://lastfm-img2.akamaized.net/i/u/64s/9f25933143434b6ccb25d7214133f34e.png',
    'size': 'medium'},
   {'#text': 'https://lastfm-img2.akamaized.net/i/u/174s/9f25933143434b6ccb25d7214133f34e.png',
    'size': 'large'},
   {'#text': 'https://lastfm-img2.akamaized.net/i/u/300x300/9f25933143434b6ccb25d7214133f34e.png',
    'size': 'extralarge'},
   {'#text': 'https://lastfm-img2.akamaized.net/i/u/300x300/9f25933143434b6ccb25d7214133f34e.png',
    'size': 'mega'},
   {'#text': 'https://lastfm-img2.akamaized.net/i/u/300x300/9f25933143434b6ccb25d7214133f34e.png',
    'size': ''}],
  'listeners': '305824',
  'playcount': '10974182',
  'tracks': {'track': [{'name': 'Legend',
     'url': 'htt

In [ ]:
#posty stuff
"""
“Paranoid”
“Spoil My Night” feat. Swae Lee
“Rich & Sad”
“Zack and Codeine”
“Takin' Shots”
“rockstar” feat. 21 Savage
“Over Now”
“Psycho” feat. Ty Dolla $ign
“Better Now”
“Ball For Me” feat. Nicki Minaj
“Otherside”
“Stay”
“Blame It On Me”
“Same Bitches” feat. G-Eazy & YG
“Jonestown (Interlude)”
“92 Explorer”
“Candy Paint”
“Sugar Wraith”
"""